# **Analyzing Students' Mental Health**
##Project Instruction <br>
Explore and analyze the students data to see how the length of stay (stay) impacts the average mental health diagnostic scores of the international students present in the study. <br>
<br>

Return a table with nine rows and five columns.
The five columns should be aliased as: stay, count_int, average_phq, average_scs, and average_as, in that order. <br>
<br>
The average columns should contain the average of the todep (PHQ-9 test), tosc (SCS test), and toas (ASISS test) columns for each length of stay, rounded to two decimal places. <br>
<br>
The count_int column should be the number of international students for each length of stay. <br>
<br>
Sort the results by the length of stay in descending order. <br>
<br>
Note: Creating new cells in the workbook will rename the DataFrame. Make sure that your final solution uses the name df.

-----------------------------------------------------

Does going to university in a different country affect your mental health? A Japanese international university surveyed its students in 2018 and published a study the following year that was approved by several ethical and regulatory boards.


---


The study found that international students have a higher risk of mental health difficulties than the general population, and that social connectedness (belonging to a social group) and acculturative stress (stress associated with joining a new culture) are predictive of depression.


---

Explore the `students` data using PostgreSQL to find out if you would come to a similar conclusion for international students and see if the length of stay is a contributing factor.


---


Here is a data description of the columns you may find helpful.

| Field Name    | Description                                      |
| ------------- | ------------------------------------------------ |
| `inter_dom`     | Types of students (international or domestic)   |
| `japanese_cate` | Japanese language proficiency                    |
| `english_cate`  | English language proficiency                     |
| `academic`      | Current academic level (undergraduate or graduate) |
| `age`           | Current age of student                           |
| `stay`          | Current length of stay in years                  |
| `todep`         | Total score of depression (PHQ-9 test)           |
| `tosc`          | Total score of social connectedness (SCS test)   |
| `toas`          | Total score of acculturative stress (ASISS test) |

# Client's questions:

Does longer stay improve mental health?

Are new international students more depressed?

Where should universities focus support?

In [ ]:
import pandas as pd
df = pd.read_csv('students.csv')
df.head()

,inter_dom,region,gender,academic,age,age_cate,stay,stay_cate,japanese,japanese_cate,...,friends_bi,parents_bi,relative_bi,professional_bi,phone_bi,doctor_bi,religion_bi,alone_bi,others_bi,internet_bi
0,Inter,SEA,Male,Grad,24,4,5,Long,3,Average,...,Yes,Yes,No,No,No,No,No,No,No,No
1,Inter,SEA,Male,Grad,28,5,1,Short,4,High,...,Yes,Yes,No,No,No,No,No,No,No,No
2,Inter,SEA,Male,Grad,25,4,6,Long,4,High,...,No,No,No,No,No,No,No,No,No,No
3,Inter,EA,Female,Grad,29,5,1,Short,2,Low,...,Yes,Yes,Yes,Yes,No,No,No,No,No,No
4,Inter,EA,Female,Grad,28,5,1,Short,1,Low,...,Yes,Yes,No,Yes,No,Yes,Yes,No,No,No


In [ ]:
import sqlite3 as sql
conn = sql.connect('students.db')
df.to_sql('students',conn,index=False, if_exists='replace')

268

In [17]:
query = """
SELECT stay, COUNT(inter_dom) AS count_int,
ROUND(AVG(todep),2) AS average_phq,
ROUND(AVG(tosc),2) AS average_scs,
ROUND(AVG(toas), 2) AS average_as
FROM students
WHERE inter_dom = 'Inter'
GROUP BY stay
ORDER BY stay DESC;
"""

pd.read_sql(query, conn)

,stay,count_int,average_phq,average_scs,average_as
0,10,1,13.00,32.00,50.00
1,8,1,10.00,44.00,65.00
2,7,1,4.00,48.00,45.00
3,6,3,6.00,38.00,58.67
4,5,1,0.00,34.00,91.00
5,4,14,8.57,33.93,87.71
6,3,46,9.09,37.13,78.00
7,2,39,8.28,37.08,77.67
8,1,95,7.48,38.11,72.80


## Does longer stay improve mental health?

**Answer**: Based on the analysis, there is no decreasing trend in depression scores with longer stay.
So, there is no clear evidence that longer stay improves students' mental health.

In [16]:
query1= """
SELECT stay, ROUND(AVG(todep), 2) AS DepressionScore
FROM students
WHERE inter_dom = 'Inter'
GROUP BY stay
ORDER BY stay DESC;
"""
pd.read_sql(query1, conn)

,stay,DepressionScore
0,10,13.00
1,8,10.00
2,7,4.00
3,6,6.00
4,5,0.00
5,4,8.57
6,3,9.09
7,2,8.28
8,1,7.48


## Are new international students more depressed?
**Answer:** Based on the trend, new international students' are not more depressed than students with longer stays. But the highest depression scores are observed among student's with longer stay. Although, trend is not entirely consistent across all durations.

## Where should universities focus support?
**Answer:** Based on the analysis, Universities should focus support on students with stay 10 first, as they show higher stress level and depression score and lowest social connectedness.

Later they should support students with a stay duration of 5 because their social connectedness is poor and stress level is high

In [19]:
df2 = pd.read_sql(query, conn)

In [21]:
df2.to_sql('students1',conn,index=False, if_exists='replace')
df2

,stay,count_int,average_phq,average_scs,average_as
0,10,1,13.00,32.00,50.00
1,8,1,10.00,44.00,65.00
2,7,1,4.00,48.00,45.00
3,6,3,6.00,38.00,58.67
4,5,1,0.00,34.00,91.00
5,4,14,8.57,33.93,87.71
6,3,46,9.09,37.13,78.00
7,2,39,8.28,37.08,77.67
8,1,95,7.48,38.11,72.80


In [25]:
query = """
SELECT stay, average_phq, average_scs, average_as
FROM students1
WHERE average_phq = (SELECT MAX(average_phq) FROM students1)
OR average_scs = (SELECT MIN(average_scs) FROM students1)
OR average_as = (SELECT MAX(average_as) FROM students1);
"""
pd.read_sql(query, conn)

,stay,average_phq,average_scs,average_as
0,10,13.0,32.0,50.0
1,5,0.0,34.0,91.0
